In [14]:
%%pyspark
df = spark.read.load('abfss://files@datalakea83eylv.dfs.core.windows.net/products/products.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(sparka83eylv, 0, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0d060041-3305-4824-9ce5-07855b44c461)

In [15]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").save(delta_table_path)

StatementMeta(sparka83eylv, 0, 16, Finished, Available, Finished)

In [27]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, delta_table_path)

deltaTable.update(
    condition= "ProductID == 772",
    set={ "ListPrice": "ListPrice * 0.3"}
)

deltaTable.toDF().show(10)


StatementMeta(sparka83eylv, 0, 28, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes| 815.9976|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [28]:
new_df = spark.read.format('delta').load(delta_table_path)
new_df.show(5)

StatementMeta(sparka83eylv, 0, 29, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes| 815.9976|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
+---------+--------------------+--------------+---------+
only showing top 5 rows



In [31]:
new_df = spark.read.format('delta').option('versionAsOf',3).load(delta_table_path)
new_df.show(5)

StatementMeta(sparka83eylv, 0, 32, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes| 815.9976|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
+---------+--------------------+--------------+---------+
only showing top 5 rows



In [32]:
 deltaTable.history(10).show(20, False, True)

StatementMeta(sparka83eylv, 0, 33, Finished, Available, Finished)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 3                                                                                                                                                                                                                                    
 timestamp           | 2025-01-05 12:05:42.513                                                                                                                                                                                                              
 userId              | null                                                                                                                                                                                                                      

In [37]:
spark.sql('CREATE DATABASE AdventureWorks')
spark.sql("CREATE TABLE AdventureWorks.ProductExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductExternal").show()

StatementMeta(sparka83eylv, 0, 38, Finished, Available, Finished)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|           ProductID|              string|   null|
|         ProductName|              string|   null|
|            Category|              string|   null|
|           ListPrice|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|                Name|spark_catalog.adv...|       |
|                Type|            EXTERNAL|       |
|            Location|abfss://files@dat...|       |
|            Provider|               delta|       |
|               Owner|trusted-service-user|       |
|    Table Properties|[delta.minReaderV...|       |
+--------------------+--------------------+-------+



In [38]:
%%sql
Use AdventureWorks;
SELECT * FROM ProductExternal;

StatementMeta(sparka83eylv, 0, 40, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [40]:
df.write.format('delta').saveAsTable("AdventureWorks.ProductManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductManaged").show()

StatementMeta(sparka83eylv, 0, 42, Finished, Available, Finished)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|           ProductID|              string|   null|
|         ProductName|              string|   null|
|            Category|              string|   null|
|           ListPrice|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|                Name|spark_catalog.adv...|       |
|                Type|             MANAGED|       |
|            Location|abfss://files@dat...|       |
|            Provider|               delta|       |
|               Owner|trusted-service-user|       |
|    Table Properties|[delta.minReaderV...|       |
+--------------------+--------------------+-------+



In [42]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductManaged;

StatementMeta(sparka83eylv, 0, 46, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [43]:
 %%sql

 USE AdventureWorks;

 SHOW TABLES;

StatementMeta(sparka83eylv, 0, 48, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 3 rows and 3 fields>

In [44]:
 %%sql

 USE AdventureWorks;

 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;

StatementMeta(sparka83eylv, 0, 51, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [45]:
%%sql
USE AdventureWorks;
CREATE TABLE Products USING DELTA LOCATION '/delta/products-delta';

StatementMeta(sparka83eylv, 0, 53, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [46]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM Products;

StatementMeta(sparka83eylv, 0, 55, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [48]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

inputPath='/data/'
mssparkutils.fs.mkdirs(inputPath)

jsonSchema = StructType([
    StructField("device", StringType(),False),
    StructField("status", StringType(), False)
])

iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)
device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
print("Source stream created...")

StatementMeta(sparka83eylv, 0, 57, Finished, Available, Finished)

Source stream created...


In [49]:
delta_stream_table_path = '/delta/iotdevicedata'
checkpointpath = '/delta/checkpoint'
deltastream = iotstream.writeStream.format('delta').option('checkpointLocation', checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(sparka83eylv, 0, 58, Finished, Available, Finished)

Streaming to delta sink...


In [50]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)

StatementMeta(sparka83eylv, 0, 59, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c803c4bb-7ffc-47d2-b6a5-d84505bf9f47)

In [51]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(sparka83eylv, 0, 60, Finished, Available, Finished)

DataFrame[]

In [52]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparka83eylv, 0, 61, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [55]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(sparka83eylv, 0, 64, Finished, Available, Finished)

True

In [56]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparka83eylv, 0, 65, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [57]:
 deltastream.stop()

StatementMeta(sparka83eylv, 0, 66, Finished, Available, Finished)